# Preview - Finite Elements in 1D

This notebook implements an example of a FEM in 1D, following the course notes, for the BVP 

$$
  - u'' = f, \quad x \in (0, 1), \qquad u(0) = u(1) = 0. 
$$

The FE space / approximation space is defined as 

$$
   V_h = \{ v_h \in C([0, 1]) : \text{p.w. affine w.r.t. } (x_i)_i, v_h(0) = v_h(1) = 0 \}
$$

where $x_0 < x_1 < \dots < x_N$ is the grid, and the FEM in variational form as 

$$ 
   \int u_h' v_h' dx = \int f v_h dx \qquad \forall v_h \in V_h. 
$$

After expanding 

$$ 
   u_h = \sum_i U_i \psi_i, \qquad v_h = \sum_i V_i \psi_i, 
$$

we obtain, equivalently, 

$$ 
\begin{aligned} 
     V^T A U &= V^T F \qquad \forall V \in \mathbb{R}^{N-1} \\ 
     A_{ij} &= \int \psi_j' \psi_i' dx   \\ 
     F_j &= \int f \psi_j dx
\end{aligned}
$$

and this can be solved via 

$$
    A U = F. 
$$

In [ ]:
using Pkg; Pkg.activate(".")
using LinearAlgebra, SparseArrays, CairoMakie

In [ ]:
"""
Assemble the FE system, i.e. the matrix A and the vector F, using mid-point quadrature to evaluate F_j. 

Inputs: 
- `X` : a vector of grid points, [x0, x1, ...]
- `f` : the function f defining the right-hand side of the BVP.
"""
function assemble(X, f)
   N = length(X)-1 
   # allocate arrays for the system matrix A and RHS F. 
   # For A we should use a sparse datastructure - we will do this later.
   F = zeros(N+1)
   A = zeros(N+1, N+1)  
    
   # FE assembly works as a loop over elements (xᵢ, xᵢ₊₁) 
   # (note the 1-based indexing)
   for i = 1:N 
      hᵢ = X[i+1] - X[i]
      # use midpoint quadrature to assemble rhs 
      # ( note that ψᵢ(ξᵢ) = ψᵢ₊₁ = 0.5 )
      ξᵢ = (X[i+1] + X[i])/2
      F[i] += hᵢ * f(ξᵢ) * 0.5 
      F[i+1] += hᵢ * f(ξᵢ) * 0.5
      # assemble stiffness matrix, for derivation see class notes.
      A[i,i] += 1/hᵢ
      A[i,i+1] += -1/hᵢ
      A[i+1,i] += -1/hᵢ
      A[i+1,i+1] += 1/hᵢ
   end
   
   # we assembled A a dense, now convert it to sparse 
   # for efficient solution of the linear system
   # In real problems we should of course also assemble 
   # in a sparse format (normally triplet...)
   return sparse(A), F 
end


In [ ]:
# define a problem with f(x) = 1, 11 gridpoints.
N = 10 
f = x -> 1 

# solve
X = range(0, 1, length = N+1) 
A, F = assemble(X, f)
U = zeros(N+1)
U[2:end-1] = A[2:end-1, 2:end-1] \ F[2:end-1]; 

In [ ]:
# plotting should be done in a separate cell. Here, for 
# 1D it doesn't matter, but sometimes the solution can take 
# a long time and should be done only once, the plotting is then 
# part of the post-processing.

fig = Figure(size = (500, 250))
ax = Axis(fig[1, 1], xlabel = "x", ylabel = "u(x)")
xp = range(0, 1, length=100)
lines!(ax, xp, 0.5 .* xp .* (1 .- xp), label = "exact")
scatterlines!(ax, X, U, color = :red, label = "FEM")
Legend(fig[1, 2], ax)
fig

In [ ]:
# The system matrix is just the standard centered FD operator!
A

In [ ]:
# The key difference is that the code works without changes 
# with an irregular grid
Xirreg = collect(X)
Xirreg[2:N] += 0.66/N * (rand(N-1) .- 0.5)
A, F = assemble(Xirreg, f)
U = zeros(N+1)
U[2:end-1] = A[2:end-1, 2:end-1] \ F[2:end-1];

In [ ]:
fig = Figure(size = (500, 250))
ax = Axis(fig[1, 1], xlabel = "x", ylabel = "u(x)")
xp = range(0, 1, length=100)
lines!(ax, xp, 0.5 .* xp .* (1 .- xp), label = "exact")
scatterlines!(ax, Xirreg, U, color = :red, label = "FEM")
Legend(fig[1, 2], ax)
fig

Why is this useful? It gives us a lot of additional flexibility which is particularly important in dimension > 1. But even in 1D we can give a simple example that shows this. Here we also see the method of manufactured solutions for the first time: suppose the exact solution of the problem is given by 

$$
u(x) = x - \frac{1 - \exp(a*x)}{1 - \exp(a)}
$$

which has an exponential boundary layer:

In [ ]:
u = x -> x - (1 - exp(30*x)) / (1 - exp(30))

fig = Figure(size = (500, 250)); ax = Axis(fig[1, 1], xlabel = "x", ylabel = "u(x)")
xp = range(0, 1, length=100)
lines!(ax, xp, u.(xp), label = "exact")
fig

We can get the applied force that results in this solution via differentiation.

In [ ]:
using ForwardDiff
du = x -> ForwardDiff.derivative(u, x)
f_a = x -> - ForwardDiff.derivative(du, x)

The concentration of the features near the right-hand boundary suggests that we should use a staggered grid. 

In [ ]:
# solution with uniform mesh 
N = 12
X1 = range(0, 1, length = N+1) 
A, F = assemble(X1, f_a)
U1 = zeros(N+1)
U1[2:end-1] = A[2:end-1, 2:end-1] \ F[2:end-1]; 

# solution with staggered grid 
X2 = sort([ 1 .- logrange(1e-2, 1.0, length=N); [1.0] ])
A, F = assemble(X2, f_a)
U2 = zeros(N+1)
U2[2:end-1] = A[2:end-1, 2:end-1] \ F[2:end-1];

In [ ]:
fig = Figure(size = (500, 250))
ax = Axis(fig[1, 1], xlabel = "x", ylabel = "u(x)")
xp = range(0, 1, length=100)
lines!(ax, xp, u.(xp), label = "exact")
scatterlines!(ax, X1, U1, color = :orange, label = "FEM-uniform")
scatterlines!(ax, X2, U2, color = :red, label = "FEM-staggered")
Legend(fig[1, 2], ax)
fig